# Soybean TTA: Comprehensive Results Summary

## Overview

This notebook aggregates and visualizes results from all TTA experiments:
- **2 Base Models**: ConvNeXt Tiny, ViT Base Patch16
- **5 TTA Methods**: BN+Pseudo, MEMO, TENT, EATA, Ensemble
- **Cross-domain task**: ASDID (source) → MH (target)

## Setup

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

sys.path.insert(0, os.path.abspath('../'))

from src.utils.plotting import plot_confusion_matrix_grid

RESULTS_DIR = Path('results')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Experiment Registry

In [ ]:
# Load all experiment results
registry_path = RESULTS_DIR / 'experiment_registry.csv'
df = pd.read_csv(registry_path)

print(f"Total experiments logged: {len(df)}")
print(f"\nExperiment breakdown:")
print(df.groupby(['model', 'dataset', 'method']).size())

# Display full registry
df

## Filter MH Cross-Domain Results

In [ ]:
# Filter for MH cross-domain results only
mh_results = df[df['dataset'] == 'MH'].copy()

# Convert to numeric
for col in ['accuracy', 'precision', 'recall', 'f1']:
    mh_results[col] = pd.to_numeric(mh_results[col])

# Display
mh_results[['model', 'method', 'accuracy', 'precision', 'recall', 'f1']].sort_values(['model', 'f1'], ascending=[True, False])

## Comparison Table: CNN vs ViT

In [ ]:
# Pivot table for easy comparison
comparison = mh_results.pivot_table(
    index='method',
    columns='model',
    values=['f1', 'accuracy'],
    aggfunc='mean'
).round(4)

print("\n" + "="*80)
print("F1 Score Comparison: ConvNeXt Tiny vs ViT Base Patch16".center(80))
print("="*80)
print(comparison['f1'])
print("\n" + "="*80)
print("Accuracy Comparison".center(80))
print("="*80)
print(comparison['accuracy'])

comparison

## F1 Score Comparison Chart

In [ ]:
# Create grouped bar chart
fig, ax = plt.subplots(figsize=(14, 7))

# Prepare data
methods = mh_results['method'].unique()
cnn_f1s = [mh_results[(mh_results['method'] == m) & (mh_results['model'] == 'convnext_tiny')]['f1'].values[0] if len(mh_results[(mh_results['method'] == m) & (mh_results['model'] == 'convnext_tiny')]) > 0 else 0 for m in methods]
vit_f1s = [mh_results[(mh_results['method'] == m) & (mh_results['model'] == 'vit_b_16')]['f1'].values[0] if len(mh_results[(mh_results['method'] == m) & (mh_results['model'] == 'vit_b_16')]) > 0 else 0 for m in methods]

x = np.arange(len(methods))
width = 0.35

bars1 = ax.bar(x - width/2, cnn_f1s, width, label='ConvNeXt Tiny', color='#2E86AB', alpha=0.8)
bars2 = ax.bar(x + width/2, vit_f1s, width, label='ViT Base Patch16', color='#A23B72', alpha=0.8)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                    f'{height:.4f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

ax.set_xlabel('TTA Method', fontsize=12, fontweight='bold')
ax.set_ylabel('F1 Score', fontsize=12, fontweight='bold')
ax.set_title('F1 Score Comparison: ConvNeXt vs ViT Across TTA Methods', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(methods, rotation=45, ha='right')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, max(max(cnn_f1s), max(vit_f1s)) * 1.15)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'cnn_vit_tta_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## F1 Gains from Baseline

In [ ]:
# Calculate F1 gains
cnn_baseline = mh_results[(mh_results['model'] == 'convnext_tiny') & (mh_results['method'] == 'baseline')]['f1'].values[0]
vit_baseline = mh_results[(mh_results['model'] == 'vit_b_16') & (mh_results['method'] == 'baseline')]['f1'].values[0]

mh_results['f1_gain'] = mh_results.apply(
    lambda row: row['f1'] - cnn_baseline if row['model'] == 'convnext_tiny' else row['f1'] - vit_baseline,
    axis=1
)

# Display gains
gains_df = mh_results[mh_results['method'] != 'baseline'][['model', 'method', 'f1', 'f1_gain']].sort_values(['model', 'f1_gain'], ascending=[True, False])

print("\n" + "="*70)
print("F1 Gains from Baseline".center(70))
print("="*70)
print(f"ConvNeXt Baseline: {cnn_baseline:.4f}")
print(f"ViT Baseline:      {vit_baseline:.4f}")
print("="*70)
gains_df

## F1 Gains Visualization)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# CNN gains
cnn_gains = mh_results[(mh_results['model'] == 'convnext_tiny') & (mh_results['method'] != 'baseline')]
colors_cnn = ['green' if g > 0 else 'red' for g in cnn_gains['f1_gain']]

ax1.barh(cnn_gains['method'], cnn_gains['f1_gain'], color=colors_cnn, alpha=0.7, edgecolor='black')
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2)
ax1.set_xlabel('F1 Gain', fontsize=12, fontweight='bold')
ax1.set_title('ConvNeXt Tiny: F1 Gains from Baseline', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# ViT gains
vit_gains = mh_results[(mh_results['model'] == 'vit_b_16') & (mh_results['method'] != 'baseline')]
colors_vit = ['green' if g > 0 else 'red' for g in vit_gains['f1_gain']]

ax2.barh(vit_gains['method'], vit_gains['f1_gain'], color=colors_vit, alpha=0.7, edgecolor='black')
ax2.axvline(x=0, color='black', linestyle='--', linewidth=2)
ax2.set_xlabel('F1 Gain', fontsize=12, fontweight='bold')
ax2.set_title('ViT Base Patch16: F1 Gains from Baseline', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'f1_gains_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## Best Performing Methods

In [ ]:
# Find best methods for each model
cnn_best = mh_results[mh_results['model'] == 'convnext_tiny'].nlargest(1, 'f1')
vit_best = mh_results[mh_results['model'] == 'vit_b_16'].nlargest(1, 'f1')

print("\n" + "="*80)
print("🏆 BEST PERFORMING METHODS".center(80))
print("="*80)

print("\nConvNeXt Tiny:")
print(f"  Method: {cnn_best['method'].values[0]}")
print(f"  F1 Score: {cnn_best['f1'].values[0]:.4f}")
print(f"  Gain from baseline: +{cnn_best['f1_gain'].values[0]:.4f}")

print("\nViT Base Patch16:")
print(f"  Method: {vit_best['method'].values[0]}")
print(f"  F1 Score: {vit_best['f1'].values[0]:.4f}")
print(f"  Gain from baseline: +{vit_best['f1_gain'].values[0]:.4f}")

# Overall best
overall_best = mh_results.nlargest(1, 'f1')
print("\n" + "-"*80)
print("🥇 OVERALL BEST CONFIGURATION".center(80))
print("-"*80)
print(f"  Model: {overall_best['model'].values[0]}")
print(f"  Method: {overall_best['method'].values[0]}")
print(f"  F1 Score: {overall_best['f1'].values[0]:.4f}")
print(f"  Accuracy: {overall_best['accuracy'].values[0]:.4f}")
print("="*80)

## Method Rankings

In [ ]:
# Rank methods by average F1 across both models
method_avg = mh_results.groupby('method').agg({
    'f1': ['mean', 'std'],
    'accuracy': 'mean'
}).round(4)

method_avg.columns = ['F1 Mean', 'F1 Std', 'Accuracy Mean']
method_avg = method_avg.sort_values('F1 Mean', ascending=False)

print("\n" + "="*60)
print("Method Rankings (Averaged Across Models)".center(60))
print("="*60)
method_avg

## Export Summary

In [ ]:
# Export comprehensive summary
summary = {
    'experiment_summary': {
        'total_experiments': len(df),
        'models': df['model'].unique().tolist(),
        'datasets': df['dataset'].unique().tolist(),
        'methods': df['method'].unique().tolist()
    },
    'baselines': {
        'convnext_asdid': df[(df['model'] == 'convnext_tiny') & (df['dataset'] == 'ASDID') & (df['method'] == 'baseline')]['f1'].values[0] if len(df[(df['model'] == 'convnext_tiny') & (df['dataset'] == 'ASDID') & (df['method'] == 'baseline')]) > 0 else None,
        'convnext_mh': cnn_baseline,
        'vit_asdid': df[(df['model'] == 'vit_b_16') & (df['dataset'] == 'ASDID') & (df['method'] == 'baseline')]['f1'].values[0] if len(df[(df['model'] == 'vit_b_16') & (df['dataset'] == 'ASDID') & (df['method'] == 'baseline')]) > 0 else None,
        'vit_mh': vit_baseline
    },
    'best_tta': {
        'convnext': {
            'method': cnn_best['method'].values[0],
            'f1': float(cnn_best['f1'].values[0]),
            'gain': float(cnn_best['f1_gain'].values[0])
        },
        'vit': {
            'method': vit_best['method'].values[0],
            'f1': float(vit_best['f1'].values[0]),
            'gain': float(vit_best['f1_gain'].values[0])
        },
        'overall': {
            'model': overall_best['model'].values[0],
            'method': overall_best['method'].values[0],
            'f1': float(overall_best['f1'].values[0])
        }
    }
}

import json
with open(RESULTS_DIR / 'cnn_vit_tta_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✅ Summary exported to cnn_vit_tta_summary.json")
print(json.dumps(summary, indent=2))

## Key Findings

In [ ]:
print("\n" + "#"*80)
print("# KEY FINDINGS".ljust(79) + "#")
print("#"*80)

print("\n1. BASELINE PERFORMANCE:")
print(f"   - ConvNeXt: ASDID F1={summary['baselines']['convnext_asdid']:.4f} → MH F1={cnn_baseline:.4f} (drop: {summary['baselines']['convnext_asdid'] - cnn_baseline:.4f})")
print(f"   - ViT:      ASDID F1={summary['baselines']['vit_asdid']:.4f} → MH F1={vit_baseline:.4f} (drop: {summary['baselines']['vit_asdid'] - vit_baseline:.4f})")

print("\n2. TTA EFFECTIVENESS:")
print(f"   - ConvNeXt best: {summary['best_tta']['convnext']['method']} (+{summary['best_tta']['convnext']['gain']:.4f} F1)")
print(f"   - ViT best:      {summary['best_tta']['vit']['method']} (+{summary['best_tta']['vit']['gain']:.4f} F1)")

print("\n3. OVERALL WINNER:")
print(f"   - {summary['best_tta']['overall']['model']} + {summary['best_tta']['overall']['method']}: F1={summary['best_tta']['overall']['f1']:.4f}")

print("\n4. METHOD INSIGHTS:")
for idx, (method, row) in enumerate(method_avg.iterrows(), 1):
    print(f"   {idx}. {method}: Avg F1={row['F1 Mean']:.4f} ± {row['F1 Std']:.4f}")

print("\n" + "#"*80)